# Honours Project 
## Author: Emily Cooper
## Supervisor: Geoffrey Vasil
### Topic: Discrete model simulation
This notebook is part of a honours project in applied mathematics at the University of Sydney. 

Abstract: The displacement of people in humanitarian crises has pervaded history. However, little has been done to model how such a crisis evolves as people under threat make the decision to flee. We use a discrete SEIR model to derive a partial differential equation of the number of people within a population who have fled. We then discuss the microscopic implications of our macroscopic model. By analysing through both deterministic and stochastic frameworks, we interpret characteristics of our model and its parameters on both microscopic and macroscopic levels. In particular, we discuss how our model suggests an internal utility variable that individuals carry around with them describing risks and opportunity. We speculate about how these results may conform with Rational Choice Theory or its generalisation, the Prospect Theory of Kahneman and Tversky. We then simulate fabricated crisis events through the discrete model, investigating four customisations of our generalised framework to simulate real-world scenarios. 


In [2]:
# Importing packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.integrate import quad

In [3]:
class discrete_refugee:

    def __init__(self, bins = np.linspace(0.1,1,5), num_people = 2000, p = [], danger_time = 2.5, escape_param = 0.9, with_return = 0, scope = 2, danger_form = "two_spike", escape_form = "threshold", return_form = "all_start", safety_form = "average"):
        
        # Allow random split into bins if necessary 
        if len(p) != 0:
            self.p_dis = p
        else:
            rand_state = np.random.rand(len(bins))
            self.p_dis = rand_state/np.sum(rand_state)*num_people
            self.p_dis = self.p_dis/np.sum(self.p_dis)

        # Setting up attributes of the system 
        self.bins = bins
        self.num_states = len(bins)
        self.time = 0
        self.total_pop = num_people
        self.danger_time = danger_time
        self.escape_param = escape_param
        self.scope = scope

        # Setting up functional form choices for D, S, \varphi, \gamma
        self.danger_form = danger_form
        self.escape_form = escape_form
        self.return_form = return_form
        self.safety_form = safety_form

        # Setting up integ function 

        # Quantities needed for return function 
        self.ret = with_return
        self.lag = 3
        self.first_time = True
        self.prev = 0
        self.t0 = 0

        # Initial states of each of the variables of interest 
        self.u = num_people*self.p_dis
        self.initdis = np.copy(self.u)
        self.e = np.zeros(len(bins))
        self.r = 0

        # Memory term of time
        self.e_mem = np.zeros((len(bins),3))
        self.dt = 1e-3
       
    
    def danger(self,bin):
        if self.danger_form == "two_spike":
            return 0.5*bin**2/(1 + (self.time - 3)**2) + 0.5*bin**2/(1 + (self.time - self.danger_time)**2)
        elif self.danger_form == "wiggly":
            return  self.time +  bin*np.sin(self.time*self.danger_time)  
        elif self.danger_form == "periodic":
            return np.minimum(bin*np.sin(self.danger_time*self.time)**2 + bin, 1)
        elif self.danger_form == "sigmoid":
            return np.minimum(bin/(bin + np.exp(-self.time + self.danger_time)), 1)
        elif self.danger_form == "sigmoid_pert":
            return np.maximum(np.minimum(bin/(bin + np.exp(-self.time + self.danger_time)) + np.random.normal(), 1),0)

    def safety(self,bin):
        pos = np.argmax(bin == self.bins)
        if self.safety_form == "average":
            nearest_neighbours = np.sum(self.u[max(pos - (self.scope - 1),0): min(pos + self.scope, self.num_states - 1)])/np.sum(self.initdis[max(pos - (self.scope - 1),0): min(pos + self.scope, self.num_states-1)])
            return nearest_neighbours 
        elif self.safety_form == "deriv":
            return (self.e_mem[pos,0] - 2*self.e_mem[pos,1] + self.e_mem[pos,2])/self.dt**2

    def escape(self, bin):
        if self.escape_form == "threshold":
            if bin >= self.escape_param:
                return 1
            else:
                return 0
        elif self.escape_form == "integ":
            def dang(t, bin):
                 return self.danger(bin)
            return quad(dang, 0, self.time, args = (bin, ))[0]
        elif self.escape_form == "integ_frac":
            def dang(t, bin):
                 return self.danger(bin)/(1+t**2)
            return quad(dang, 0, self.time, args = (bin, ))[0]
        elif self.escape_form == "integ_e":
             def dang(t, bin):
                 return self.danger(bin)*np.exp(-t)
             return quad(dang, 0, self.time, args = (bin, ))[0]
    
    def returns(self):
        cond = np.mean([self.danger(b) for b in self.bins]) < 0.2
        if cond and self.first_time:
            if self.time > self.lag:
                self.t0 = self.time
                self.first_time = False
            else:
                cond = 0
        elif not cond and not self.first_time:
            self.first_time = True
        if cond:
            self.prev = cond*self.r*(self.time - self.t0)*0.01
            return self.prev
        else:
            new_one = self.prev/(1+1e-3)
            self.prev = new_one
            return new_one

    def step(self, dt = 1e-3):

        # Set backwards and forwards and account for boundary conditions
        danger_roll_back = np.roll(self.danger(self.bins),shift = 1)
        safety_roll_back = np.roll(np.array([self.safety(bin) for bin in self.bins]),shift = 1)
        e_roll_forward = np.roll(self.e, shift = -1)
        e_roll_back = np.roll(self.e, shift = 1)
        danger_roll_back[0], safety_roll_back[0], e_roll_forward[-1] = 0, 0, 0


        up_change = self.danger(self.bins)*self.u
        last_step = self.danger(self.bins)*self.e

        self.r += dt*np.sum([*np.array([self.escape(bin) for bin in self.bins])*self.e, last_step[-1]]) - self.ret*self.returns()
        self.u -= dt*up_change 


        if self.return_form == "all_start":
            self.u[0] += self.ret*self.returns()
        elif self.return_form == "all_end":
            self.u[-1] += self.ret*self.returns()
        elif self.return_form == "split":
            self.u += self.ret.self.returns()/self.num_states


        self.e += dt*(up_change + danger_roll_back*e_roll_back-self.danger(self.bins)*self.e + self.safety(self.bins)*e_roll_forward  - safety_roll_back*self.e - np.array([self.escape(bin) for bin in self.bins])*self.e)
        self.e_mem = np.roll(self.e_mem, shift = 1, axis = 1)
        self.e_mem[:,0] = self.e
        self.time += dt

    def plot_safety(self,niters):
        safetys = np.zeros(niters)
        for i in range(niters):
            self.step()
            safetys[i] = np.mean(np.array([self.safety(bin) for bin in self.bins]))
        return safetys

    def move(self, niters):
        e_s = np.zeros((niters, self.num_states))
        u_s = np.zeros((niters, self.num_states))
        r = np.zeros(niters)
        for i in range(niters):
            r[i] = self.r
            u_s[i,:] = self.u
            e_s[i,:] = self.e
            self.step()
        return u_s, e_s, r